# Recap sur le modèle Bag Of Words
 - Un texte est modélisé comme un ensemble non ordonné de mots
 - Chaque mot va être représenté par un vecteur avec la technique du One-Hot Encoding
 - On peut créer différentes variations du modèle en comptant les mots de façon + ou - complexe

# Les forces du modèle Bag Of Words
- Il est simple!
- Il s'adapte à n'importe quelle langue et à n'importe quel vocabulaire
- Il y a beaucoup de variants : TF, TF-IDF, BM25
- On peut y rajouter de l'information en utilisant des N-grams
- C'est une méthode ancienne : Beaucoup de litérature sur le sujet, et des implémentations dans tout les langages de programmation

# Les limites du modèle Bag of Words
- La taille des vecteurs générés grossit avec la taille du vocabulaire 
- L'ajout des N-Grams fait exploser cette croissance
- Les vecteurs générés sont très sparses ça pose problème aux méthodes de machine learning qui les utilisent
- Des mots similaires ont une représentation différente
- C'est un peu triste de faire juste du comptage de mots...


# Les embeddings vectoriels à la rescousse

On va conserver la même approche, on veut convertir un mot en vecteur, mais sans les désagréments du modèle BoW.

La promesse des embeddings vectoriels : 
- Chaque mot va être représenté par un *vecteur dense* de taille *raisonnable*
- Ce vecteur va être porteur de sens (au sens sémantique)

Ok, mais comment?

# Le modèle qui a changé la donne : Word2Vec
```
Word2vec was created and published in 2013 by a team of researchers led by Tomas Mikolov at Google
```
https://en.wikipedia.org/wiki/Word2vec

## L'idée de base : exploiter l'hypothèse distributionnelle
```
Hypothèse selon laquelle les mots qui apparaissent dans les mêmes contextes linguistiques partagent des significations similaires.
````

Dans un texte, un mot va souvent être entouré des mêmes termes.

Le contexte d'un mot c'est les mots qui entournent ce mot. On va le choisir comme les *n* termes qui précèdent le mot que l'on traite et les *n* termes qui le suivent.

### Illustration avec des brèves de comptoir : [wikipedia](https://fr.wikipedia.org/wiki/Br%C3%A8ves_de_comptoir#Exemples)
```
« À la naissance le nain est normal, c'est en grandissant qu'il rapetisse. »
```
Contexte pour **nain** de taille 4 : [À, la, naissance, le, est, normal, c', est] 

```
« Les auteurs modernes font des livres tellement petits qu'on ne peut plus mettre des fleurs à sécher dedans. »
```
Contexte pour **livres** de taille 5 : [Les, auteurs, modernes, font, des, tellement, petits, qu',on, ne]

## Et on en fait quoi de cette idée ?

Si on arrive à faire prédire le contexte d'un mot grace à un modèle de machine learning, c'est que le modèle aurra réussi à capturer la signification du mot!

## Promesse tenue?

Google a publié un modèle Word2Vec : https://code.google.com/archive/p/word2vec/

```
The model was trained on the Google News dataset (about 100 billion words). 
The model contains 300-dimensional vectors for 3 million words and phrases.
```

Le résultat marquant :
$$\vec{V}_{King} -\vec{V}_{Man} + \vec{V}_{Woman} \approx \vec{V}_{Queen}$$

In [1]:
%%time
import gensim
w2v = gensim.models.KeyedVectors.load_word2vec_format(
    '/mnt/data/GoogleNews-vectors-negative300.bin.gz',
    binary=True,
)

CPU times: user 46.3 s, sys: 2.8 s, total: 49.1 s
Wall time: 48.9 s


In [2]:
w2v.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7118193507194519),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321839332581),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593831062317),
 ('monarchy', 0.5087411999702454)]

In [3]:
queen = w2v["king"] - w2v["man"] + w2v["woman"]

In [4]:
w2v.most_similar(positive=[queen])

[('king', 0.8449392318725586),
 ('queen', 0.7300517559051514),
 ('monarch', 0.645466148853302),
 ('princess', 0.6156251430511475),
 ('crown_prince', 0.5818676352500916),
 ('prince', 0.5777117609977722),
 ('kings', 0.5613663792610168),
 ('sultan', 0.5376775860786438),
 ('Queen_Consort', 0.5344247817993164),
 ('queens', 0.5289887189865112)]

In [5]:
queen_norm = (
    w2v.get_vector("king", norm=True)
    - w2v.get_vector("man", norm=True)
    + w2v.get_vector("woman", norm=True)
)

In [6]:
w2v["queen"][:10], queen[:10], queen_norm[:10]

(array([ 0.00524902, -0.14355469, -0.06933594,  0.12353516,  0.13183594,
        -0.08886719, -0.07128906, -0.21679688, -0.19726562,  0.05566406],
       dtype=float32),
 array([ 0.04296875, -0.17822266, -0.12908936,  0.11523438,  0.00268555,
        -0.10229492,  0.19580078, -0.1795044 ,  0.01953125,  0.40991974],
       dtype=float32),
 array([-0.00619015, -0.07542217, -0.05083442,  0.04359096, -0.00321992,
        -0.035044  ,  0.08129873, -0.06163806, -0.00235739,  0.14262468],
       dtype=float32))

In [7]:
w2v.most_similar(positive=[queen_norm])

[('king', 0.7992597818374634),
 ('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321839332581),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.518113374710083),
 ('sultan', 0.5098593831062317)]

In [8]:
w2v.most_similar(positive=['dog', 'child'], negative=['adult'])

[('puppy', 0.672011137008667),
 ('pooch', 0.640053927898407),
 ('pup', 0.612525224685669),
 ('dogs', 0.6005871295928955),
 ('German_shepherd', 0.591395378112793),
 ('cat', 0.5822558999061584),
 ('golden_retriever', 0.5778035521507263),
 ('pit_bull', 0.5748783349990845),
 ('Rottweiler', 0.5717918276786804),
 ('Yorkshire_terrier', 0.5693223476409912)]

In [9]:
w2v.most_similar(positive=['cat', 'child'], negative=['adult'])

[('puppy', 0.6052532196044922),
 ('kitten', 0.6021385788917542),
 ('dog', 0.5852227807044983),
 ('pup', 0.5823509693145752),
 ('pooch', 0.5800433158874512),
 ('cats', 0.5533753633499146),
 ('stray_cat', 0.5414088368415833),
 ('kitties', 0.5325700044631958),
 ('Yorkshire_terrier', 0.5320756435394287),
 ('pet', 0.5289697647094727)]

In [10]:
w2v.most_similar(positive=['meow', 'dog'], negative=["cat"])

[('woof_woof', 0.5396372079849243),
 ('dogs', 0.4995119869709015),
 ('bark_incessantly', 0.4923662543296814),
 ('Woof_woof', 0.4894609749317169),
 ('Rusty_barked', 0.4648567736148834),
 ('woofs', 0.46396562457084656),
 ('yip_yip', 0.45828351378440857),
 ('growl', 0.45362916588783264),
 ('moos', 0.4518892765045166),
 ('barking_incessantly', 0.45174774527549744)]

In [11]:
w2v.most_similar(positive=['Paris', 'Italy'], negative=["France"])

[('Milan', 0.7222141623497009),
 ('Rome', 0.702830970287323),
 ('Palermo_Sicily', 0.5967570543289185),
 ('Italian', 0.5911272764205933),
 ('Tuscany', 0.5632812976837158),
 ('Bologna', 0.5608358383178711),
 ('Sicily', 0.5596384406089783),
 ('Bologna_Italy', 0.5470058917999268),
 ('Berna_Milan', 0.5464027523994446),
 ('Genoa', 0.5308900475502014)]

In [12]:
w2v.most_similar(positive=['Bordeaux', 'Italy'], negative=["France"])

[('Friuli', 0.6579111814498901),
 ('Lecce', 0.6504460573196411),
 ('Milan', 0.6464554667472839),
 ('Livorno', 0.6451722979545593),
 ('Cagliari', 0.6402646899223328),
 ('Brescia', 0.6326268315315247),
 ('Bologna', 0.6313794851303101),
 ('Genoa', 0.6303693652153015),
 ('Tuscany', 0.6288278698921204),
 ('Palermo', 0.6284286975860596)]

In [13]:
w2v.most_similar(positive=['bordeaux', 'Italy'], negative=["France"])

[('Falanghina', 0.6475211977958679),
 ('Chianti', 0.6175957918167114),
 ('Chianti_Classico', 0.6059287786483765),
 ('Barolo', 0.6030843257904053),
 ("Nero_d'_Avola", 0.602142333984375),
 ('barbera', 0.6005603075027466),
 ('Pinot_Grigio', 0.5974807143211365),
 ('Valpolicella', 0.5972704887390137),
 ('Riserva', 0.5970005393028259),
 ('malvasia', 0.5907768607139587)]

In [14]:
w2v.most_similar(positive=['bordeaux', 'South_Africa'], negative=["France"])

[('pinotage', 0.5918348431587219),
 ('Chenin', 0.5434677004814148),
 ('shiraz', 0.5277871489524841),
 ('Vergelegen', 0.5246654748916626),
 ('Pinotage', 0.5209116339683533),
 ('rooibos_tea', 0.5206111669540405),
 ('chenin', 0.5097901821136475),
 ('Cap_Classique', 0.5078455209732056),
 ('sauvignon', 0.507658839225769),
 ('Nederburg', 0.5071874260902405)]

## Comment ça marche ?
 - Une prédiction : 2 possibilités
 - Architecture de réseau de neurone
 - La tâche fantôme
 - Principes d'entrainement

On a 2 possibilités pour la prédiction : CBOW et Skip-Gram
![Word2Vec](w2v.png)

CBOW is faster while skip-gram does a better job for infrequent words.

## Un réseau de neurones à 1 couche cachée pour faire le job
![Architecture](https://israelg99.github.io/images/2017-03-23-Word2Vec-Explained/skip_gram_net_arch.png)

## On avait parlé de générer des vecteurs !?
Et on se retrouve avec un réseau de neurones

## Magic trick :
On va jetter purement et simplement la dernière couche.

La couche cachée est en fait une matrice de taille $n_{vocab} \times n_{hidden}$

La couche d'entrée est alimentée par des mots représentés par des vecteurs "One-Hot encoded":

![embed](https://israelg99.github.io/images/2017-03-23-Word2Vec-Explained/matrix_mult_w_one_hot.png)

## Voilà notre embedding
![MagicTrick](https://israelg99.github.io/images/2017-03-23-Word2Vec-Explained/word2vec_weight_matrix_lookup_table.png)

# Et pour entrainer le modèle on fait comment?

On utilise une méthode appelée le **négative sampling**

On génère tout d'abord des couples de mots avec le **mot à prédire** et **les mots du contexte**
![Sampling](https://israelg99.github.io/images/2017-03-23-Word2Vec-Explained/training_data.png)

On sample aussi **N mots** du vocabulaire **absent du contexte**.

On va alors maximiser la métrique (log-likelihood) pour les mots du contexte et minimiser la métrique pour les mots hors contexte.

Références :
- https://arxiv.org/abs/1301.3781
- https://en.wikipedia.org/wiki/Word2vec
- https://israelg99.github.io/2017-03-23-Word2Vec-Explained/

# TP
2 tâches : 
 - Explorer le modèle Word2Vec publié par google
 - Utiliser le modèle Word2Vec pour faire de l'analyse de sentiment sur des critiques de films